In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)

### Extract the crowdfunding.xlsx Data

In [ ]:
# Read the data into a Pandas DataFrame
crowdfunding_info_df = pd.read_excel('Resources/crowdfunding.xlsx')
crowdfunding_info_df.head()

In [ ]:
# Get a brief summary of the crowdfunding_info DataFrame.
crowdfunding_info_df.info()

### Create the Category and Subcategory DataFrames
---
**Create a Category DataFrame that has the following columns:**
* A "category_id" column that has entries going sequentially from "cat1" to "cat*n*", where *n* is the number of unique categories
* A "category" column that contains only the category titles

Export the DataFrame as a `category.csv` CSV file.

**Create a SubCategory DataFrame that has the following columns:**
* A "subcategory_id" column that has entries going sequentially from "subcat1" to "subcat*n*", where *n* is the number of unique subcategories
* A "subcategory" column that contains only the subcategory titles

Export the DataFrame as a `subcategory.csv` CSV file.

In [ ]:
# Get the crowdfunding_info_df columns.
crowdfunding_info_df.columns

In [ ]:
# Assign the category and subcategory values to category and subcategory columns.
crowdfunding_info_df[['category','subcategory']] =  \
crowdfunding_info_df['category & sub-category'].str.split('/', n=2, expand =True)
crowdfunding_info_df.head()

In [ ]:
# Get the unique categories and subcategories in separate lists.

categories = crowdfunding_info_df['category'].unique()
subcategories = crowdfunding_info_df['subcategory'].unique()

print(categories)
print(subcategories)

In [ ]:
# Get the number of distinct values in the categories and subcategories lists.

print(len(categories))
print(len(subcategories))

In [ ]:
# Create numpy arrays from 1-9 for the categories and 1-24 for the subcategories.
category_ids = np.arange(1, 10)
subcategory_ids = np.arange(1, 25)

print(category_ids)
print(subcategory_ids)

In [ ]:
# Use a list comprehension to add "cat" to each category_id. 
cat_ids  = []
for x in category_ids :
    cat_ids.append (f' cat{x}')
# Use a list comprehension to add "subcat" to each subcategory_id.    
subcat_ids  = []
for x in subcategory_ids :
    subcat_ids.append (f'subcat{x}')
    
print(cat_ids)
print(subcat_ids)

In [ ]:
# Create a category DataFrame with the category_id array as the category_id and categories list as the category name.
category_df = pd.DataFrame ({'category_id': cat_ids,'category':categories})
                              
# Create a category DataFrame with the subcategory_id array as the subcategory_id and subcategories list as the subcategory name. 
subcategory_df = pd.DataFrame ({'subcategory_id': subcat_ids,'subcategory':subcategories})
                              

In [ ]:
category_df

In [ ]:
subcategory_df

In [ ]:
# Export categories_df and subcategories_df as CSV files.
category_df.to_csv("Resources/category.csv", index=False)

subcategory_df.to_csv("Resources/subcategory.csv", index=False)

### Campaign DataFrame
----
**Create a Campaign DataFrame that has the following columns:**
- The "cf_id" column.
- The "contact_id" column.
- The “company_name” column.
- The "blurb" column is renamed as "description."
- The "goal" column.
- The "goal" column is converted to a `float` datatype.
- The "pledged" column is converted to a `float` datatype. 
- The "backers_count" column. 
- The "country" column.
- The "currency" column.
- The "launched_at" column is renamed as "launch_date" and converted to a datetime format. 
- The "deadline" column is renamed as "end_date" and converted to a datetime format.
- The "category_id" with the unique number matching the “category_id” from the category DataFrame. 
- The "subcategory_id" with the unique number matching the “subcategory_id” from the subcategory DataFrame.
- And, create a column that contains the unique four-digit contact ID number from the `contact.xlsx` file.
 

Then export the DataFrame as a `campaign.csv` CSV file.


In [ ]:
# Create a copy of the crowdfunding_info_df DataFrame name campaign_df. 
campaign_df = crowdfunding_info_df.copy()
campaign_df.head()

In [ ]:
# Rename the blurb, launched_at, and deadline columns.
campaign_df.rename(columns={"blurb":"description", \
                            "launched_at":"launch_date","deadline":"end_date"},inplace=True)
campaign_df.head()

In [ ]:
# Convert the goal and pledged columns to a `float` data type.
campaign_df [['goal','pledged']] = campaign_df [['goal','pledged']].astype('float')
campaign_df

In [ ]:
# Check the datatypes
campaign_df.dtypes

In [ ]:
# Format the launched_date and end_date columns to datetime format
from datetime import datetime as dt
campaign_df["launch_date"] = pd.to_datetime(campaign_df["launch_date"], unit='s').dt.date
campaign_df["end_date"] = pd.to_datetime(campaign_df["end_date"], unit='s').dt.date

In [ ]:
campaign_df["end_date"] = pd.to_datetime(campaign_df["end_date"])
campaign_df["launch_date"] = pd.to_datetime(campaign_df["launch_date"])

In [ ]:
# Check the datatypes
campaign_df.dtypes

In [ ]:
# Merge the campaign_df with the category_df on the "category" column and 
campaign_merged_df = pd.merge(campaign_df, category_df, on="category", how="left")

# the subcategory_df on the "subcategory" column.
campaign_merged_df = pd.merge(campaign_merged_df, subcategory_df, on="subcategory", how="left")


In [ ]:
campaign_merged_df.head()

In [ ]:
# Drop unwanted columns
columns_to_drop = ['staff_pick', 'spotlight', 'category & sub-category', 'category', 'subcategory']
# Drop the specified columns
campaign_cleaned = campaign_merged_df.drop(columns=columns_to_drop)
campaign_cleaned.head()

In [ ]:
# Export the DataFrame as a CSV file. 
campaign_cleaned.to_csv("Resources/campaign.csv", index=False)

### Extract the contacts.xlsx Data.

In [ ]:
# Read the data into a Pandas DataFrame. Use the `header=2` parameter when reading in the data.
contact_info_df = pd.read_excel('Resources/contacts.xlsx', header=2)
contact_info_df.head()

### Create the Contacts DataFrame 
---
**Create a Contacts DataFrame that has the following columns:**
- A column named "contact_id"  that contains the unique number of the contact person.
- A column named "first_name" that contains the first name of the contact person.
- A column named "last_name" that contains the first name of the contact person.
- A column named "email" that contains the email address of the contact person

Then export the DataFrame as a `contacts.csv` CSV file.

### Option 1: Use Pandas to create the contacts DataFrame.

In [ ]:
# Iterate through the contact_info_df and convert each row to a dictionary.
import json
dict_values = []
column_names = []

# Iterate over the dataframe
for i, row in contact_info_df.iterrows():
    data = row[0]
    
    # Convert each row to a Python dictionary.
    dict_data = json.loads(data)
    
    # Use a list comprehension to get the keys from the converted data.
    columns = [k for k,v in dict_data.items()]
    # Use a list comprehension to get the values for each row.
    row_values = [v for k, v in dict_data.items()]
    
    # Append the keys and list values to the lists created in step 1.  
    column_names.append(columns)
    dict_values.append(row_values)

# Print out the list of values for each row.
print(dict_values)

In [ ]:
# Create a contact_info DataFrame and add each list of values, i.e., each row 
# to the 'contact_id', 'name', 'email' columns.
contact_info_df = pd.DataFrame(dict_values, columns=column_names[0])
contact_info_df.head()

In [ ]:

# Check the datatypes.
contact_info_df.info()

In [ ]:
# Create a "first_name" and "last_name" column with the first and last names from the "name" column. 
contact_info_df[['first_name', 'last_name']] = contact_info_df['name'].str.split(' ', expand=True)

contact_info_df_clean = contact_info_df.drop(['name'], axis=1)

contact_info_df_clean



In [ ]:
# Reorder the columns
contact_info_df_clean = contact_info_df_clean[['contact_id', 'first_name', 'last_name', 'email']]
contact_info_df_clean


In [ ]:
# Check the datatypes one more time before exporting as CSV file.
contact_info_df_clean.info()


In [ ]:
# Export the DataFrame as a CSV file. 
contact_info_df_clean.to_csv("Resources/contacts.csv", encoding='utf8', index=False)